# Trait `DataStructurable`

In [1]:
# import classes
from dataclasses import dataclass
from python_middlewareable import (
    RequestBase,
    MiddlewareBase,
    MiddlewareNextCallBase,
    MiddlewareableBase,
    DataStructurableRequestMixin,
    DataStructurable,
    RequestDataBase,
    ResponseDataBase,
    TransportDataBase,
)


# define the payload structure
@dataclass
class PayloadData(RequestDataBase):  # inherit from RequestDataBase
    name: str


# define the response structure
@dataclass
class ResponseData(ResponseDataBase):  # inherit from ResponseDataBase
    data: str = ""


# add the `DataStructurableRequestMixin` to the request class
@dataclass
class Request(
    DataStructurableRequestMixin[PayloadData, ResponseData, TransportDataBase],
    RequestBase,
):
    pass


# create a middleware
class OneMiddleware(MiddlewareBase[Request]):
    async def handle(
        self, request: Request, next_call: MiddlewareNextCallBase[Request]
    ) -> None:
        await next_call(request)

        request.response_data.data += request.request_data.name + " from OneMiddleware"


# add the trait mixin
class App(DataStructurable, MiddlewareableBase[Request]):
    middlewares = [OneMiddleware]


# middlewareable
app = App()

# request
request = Request(
    request_data=PayloadData(name="John"),
    response_data=ResponseData(),
    transport_data=TransportDataBase(),
)

# process request
result = await app.process_middlewares(request)

# check the result
print(result)

# output:
# Request(
#   request_data=PayloadData(name='John'),
#   response_data=ResponseData(data='John from OneMiddleware'),
#   transport_data=TransportDataBase(),
# )

Request(request_data=PayloadData(name='John'), response_data=ResponseData(data='John from OneMiddleware'), transport_data=<python_middlewareable.traits.data_structurable.dtos.transport_data.TransportData object at 0x78339c308e90>)
